In [2]:
import torch as nn
import numpy as np
import pandas as pd

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

class SingleScoreCNN(nn.Module):
    def __init__(self):
        super(SingleScoreCNN, self).__init__()
        # Define the layers of the CNN
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 64 * 8)  # Adjust the size based on the input size
        self.fc2 = nn.Linear(64 * 8, 64 * 2)  
        self.fc3 = nn.Linear(64 * 2, 64) 
        self.fc4 = nn.Linear(64, 1)
    
    def forward(self, x):
        # Define the forward pass
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return x

# Example usage:
model = SingleScoreCNN()
output = model(torch.randn(1, 3, 64, 64))  # Example input tensor (batch_size, channels, height, width)

# Define the loss and optimizer
criterion = nn.MSELoss()  # For example, if it's a regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class MeasuresDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None, indices=None):
        data = pd.read_csv(csv_file)
        if indices is not None:
            self.data = data.iloc[indices]
        else:
            self.data = data
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        page = str(int(self.data.iloc[idx]['Page']))
        label = self.data.iloc[idx]['Final Score']

        img_path = os.path.join(self.image_dir, f"{page}.jpg")
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to the size expected by the network
    transforms.ToTensor()
])

# Load dataset
dataset = MeasuresDataset(csv_file='measures_context.csv', image_dir='./combined', transform=transform)

# Splitting data indices for train and test
total_size = len(dataset)
testsizepercent = 0.2
test_size = int(testsizepercent * total_size)
train_size = total_size - test_size

indices = torch.randperm(total_size).tolist()
train_indices = indices[test_size:]
test_indices = indices[:test_size]

train_dataset = MeasuresDataset(csv_file='measures_context.csv', image_dir='./combined', transform=transform, indices=train_indices)
test_dataset = MeasuresDataset(csv_file='measures_context.csv', image_dir='./combined', transform=transform, indices=test_indices)

# Create data loaders
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



[822, 1176, 1170, 110, 867, 1174, 364, 702, 1013, 281, 601, 247, 136, 872, 531, 1024, 500, 1069, 628, 1323, 777, 694, 12, 720, 117, 305, 1177, 39, 1031, 1332, 273, 899, 634, 360, 438, 1077, 737, 1062, 716, 492, 620, 1088, 431, 194, 1201, 1336, 836, 1020, 321, 1305, 465, 457, 445, 80, 138, 2, 866, 886, 891, 997, 1058, 250, 1068, 1128, 839, 534, 411, 953, 1075, 903, 1250, 1317, 506, 389, 549, 838, 1071, 365, 665, 1168, 524, 587, 8, 391, 1211, 1186, 738, 1283, 637, 412, 93, 1275, 1304, 501, 1216, 596, 76, 717, 242, 914, 672, 319, 315, 265, 578, 902, 132, 241, 909, 608, 423, 22, 225, 181, 153, 374, 405, 1303, 1276, 650, 519, 874, 394, 462, 303, 757, 515, 1163, 655, 409, 1106, 1040, 1264, 1067, 856, 489, 351, 257, 640, 144, 57, 662, 1131, 62, 871, 1284, 295, 338, 906, 1225, 567, 83, 956, 1125, 1270, 402, 1079, 279, 204, 193, 629, 823, 612, 484, 522, 7, 232, 1237, 1006, 240, 399, 930, 908, 253, 742, 69, 356, 432, 1219, 262, 599, 920, 699, 1318, 306, 418, 878, 663, 255, 854, 342, 463, 235, 55

In [4]:
import torch
import torch.optim as optim
import torch.nn as nn

# Prepare model, loss function, and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, labels in dataloader:
        # Move data to GPU if available
        # images, labels = images.cuda(), labels.cuda() if torch.cuda.is_available() else (images, labels)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        epoch_loss += loss.item()
    
    # Compute and print the average loss for the epoch
    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], MSE Loss: {avg_loss:.4f}")

Epoch [1/20], MSE Loss: 0.0756
Epoch [2/20], MSE Loss: 0.0183
Epoch [3/20], MSE Loss: 0.0142
Epoch [4/20], MSE Loss: 0.0141
Epoch [5/20], MSE Loss: 0.0147
Epoch [6/20], MSE Loss: 0.0152
Epoch [7/20], MSE Loss: 0.0140
Epoch [8/20], MSE Loss: 0.0132
Epoch [9/20], MSE Loss: 0.0127
Epoch [10/20], MSE Loss: 0.0128
Epoch [11/20], MSE Loss: 0.0149
Epoch [12/20], MSE Loss: 0.0141
Epoch [13/20], MSE Loss: 0.0172
Epoch [14/20], MSE Loss: 0.0163
Epoch [15/20], MSE Loss: 0.0123
Epoch [16/20], MSE Loss: 0.0134
Epoch [17/20], MSE Loss: 0.0139
Epoch [18/20], MSE Loss: 0.0111
Epoch [19/20], MSE Loss: 0.0103
Epoch [20/20], MSE Loss: 0.0111


In [11]:
torch.save(model,"simple_model.pth")

In [14]:
model = torch.load("simple_model.pth")

In [15]:
model

SingleScoreCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
)